In [3]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'

In [4]:
train = np.loadtxt(os.path.join(dir_, 'rating_matrix.csv'), delimiter=',')
pd_train = pd.read_pickle(os.path.join(dir_, 'train_normalized_to_rating_filter_track_20_user_100.pkl'))

In [5]:
num_user = len(train)
num_track = len(train[0])
print(num_user, num_track)

945 39088


In [6]:
def pearson_correlation(u1_index, u2_index):
        result = 0.0
        user1_data = train[u1_index]
        user2_data = train[u2_index]

        rx_avg = user_average_rating(user1_data)
        ry_avg = user_average_rating(user2_data)
        sxy = common_items(user1_data, user2_data)

        top_result = 0.0
        bottom_left_result = 0.0
        bottom_right_result = 0.0
        for item in sxy:
            rxs = user1_data[item]
            rys = user2_data[item]

            top_result += (rxs - rx_avg)*(rys - ry_avg)
            bottom_left_result += pow((rxs - rx_avg), 2)
            bottom_right_result += pow((rys - ry_avg), 2)
        bottom_left_result = math.sqrt(bottom_left_result)
        bottom_right_result = math.sqrt(bottom_right_result)
        
        ################################################################
        if (bottom_left_result * bottom_right_result) == 0:
            return -2, -2 # dump the data
        ################################################################
        
        result = top_result/(bottom_left_result * bottom_right_result)
        return len(sxy), result

def user_average_rating(u):
    avg_rating = 0.0
    for i in u:
        avg_rating += i
    avg_rating /= len(u) * 1.0
    return avg_rating

def common_items(u1, u2):
    result = []
    for i in range(num_track):
        if u1[i] > 0 and u2[i] > 0:
            result.append(i)
    return result

In [8]:
sim_table = pd.DataFrame(columns=['u1', 'u2', 'common', 'pearson']) 
for i in tqdm(range(num_user)):
    for j in range(i+1, num_user):
        c, p = pearson_correlation(i,j)
        if p != -2:
            sim_table = sim_table.append({'u1': i, 'u2': j, 'common': c, 'pearson': p}, ignore_index=True)
            sim_table = sim_table.append({'u1': j, 'u2': i, 'common': c, 'pearson': p}, ignore_index=True)

In [24]:
sim_table_ = sim_table[sim_table['common']>10]
sim_table_ = sim_table_.sort_values(by=['pearson'],  ascending=False)
high_similarity = sim_table_[:int(len(sim_table_)*2/10)]
low_similarity =sim_table_[int(len(sim_table_)*8/10):]
high_similarity_v = sim_table_.iloc[int(len(sim_table_)*2/10)]['pearson']
low_similarity_v = sim_table_.iloc[int(len(sim_table_)*8/10)]['pearson']
print(high_similarity_v, low_similarity_v)

0.6993128968331564 0.4970825243256837


In [29]:
print(len(sim_table), len(sim_table_))

877044 666970


In [25]:
pairs = pd.DataFrame(columns=['uid', 'pair']) 
for i in range(num_user):
    pair = len(low_similarity[low_similarity['u1'] == i])
    pairs = pairs.append({'uid': i, 'pair': pair}, ignore_index=True)
pairs = pairs.sort_values(by=['pair'],  ascending=True)

groups = []
group_size = 5
pairs = pairs[pairs['pair'] >= (group_size-1)]
p_status = len(pairs)
pbar = tqdm(total=len(pairs)) 
while len(pairs) >= group_size:
    group = []
    pbar.update(p_status - len(pairs))
    p_status -= (p_status - len(pairs))
    uid = pairs.iloc[0]['uid']
    pair = pairs.iloc[0]['pair']
    group.append(uid)
    if pair == (group_size-1):
        uid = low_similarity[low_similarity['u1'] == uid]
        for _, row in uid.iterrows():
            group.append(int(row['u2']))
        check_sim = True
        for i in range(1, group_size):
            for j in range(i+1, group_size):
                check = low_similarity[low_similarity['u1'] == group[i]]
                check = check[check['u2'] == group[j]]
                if len(check) == 0:
                    check_sim = False
        if check_sim:
#             print(group)
            groups.append(group)
            for u in group:
                low_similarity = low_similarity[low_similarity['u1'] != u]
                low_similarity = low_similarity[low_similarity['u2'] != u]
        else:
            low_similarity = low_similarity[low_similarity['u1'] != group[0]]
            low_similarity = low_similarity[low_similarity['u2'] != group[0]]
        pairs = pd.DataFrame(columns=['uid', 'pair']) 
        for i in range(num_user):
            pair = len(low_similarity[low_similarity['u1'] == i])
            pairs = pairs.append({'uid': i, 'pair': pair}, ignore_index=True)
        pairs = pairs.sort_values(by=['pair'],  ascending=True)
        pairs = pairs[pairs['pair'] >= (group_size-1)]
    else:
        uid = low_similarity[low_similarity['u1'] == uid]
        avaliable_uid = []
        for _, row in pairs.iterrows():
            if len(uid[uid['u2'] == row['uid']]) > 0:
                avaliable_uid.append(int(row['uid']))
                
        pointer = []
        for i in range(group_size-1):
            pointer.append(i)

        while pointer[0] < (len(avaliable_uid) - (group_size - 1) + 2) and len(avaliable_uid) >= (group_size-1):
            group_ = group.copy()
            for i in range(group_size-1):
                group_.append(avaliable_uid[pointer[i]])
            check_sim = True
            for i in range(1, group_size):
                for j in range(i+1, group_size):
                    check = low_similarity[low_similarity['u1'] == group_[i]]
                    check = check[check['u2'] == group_[j]]
                    if len(check) == 0:
                        check_sim = False
            if check_sim:
#                 print('_', group_)
                groups.append(group_)
                for u in group_:
                    low_similarity = low_similarity[low_similarity['u1'] != u]
                    low_similarity = low_similarity[low_similarity['u2'] != u]
                break
            else:
                p = 0
                added = False
                while not added:
                    if (group_size-2) - p < 0:
                        pointer[0] = (len(avaliable_uid) - (group_size - 1) + 2)
                        break
                    if pointer[(group_size-2) - p] < (len(avaliable_uid) - 1 - p):
                        pointer[(group_size-2) - p] += 1
                        while p > 0:
                            p -= 1
                            if (group_size-2) - p != 0:
                                pointer[(group_size-2) - p] = pointer[(group_size-2) - p - 1] + 1
                        added = True
                    else:
                        p += 1
        if pointer[0] == (len(avaliable_uid) - (group_size - 1) + 2) or len(avaliable_uid) < (group_size-1):
            low_similarity = low_similarity[low_similarity['u1'] != group[0]]
            low_similarity = low_similarity[low_similarity['u2'] != group[0]]
        pairs = pd.DataFrame(columns=['uid', 'pair']) 
        for i in range(num_user):
            pair = len(low_similarity[low_similarity['u1']==i])
            pairs = pairs.append({'uid': i, 'pair': pair}, ignore_index=True)
        pairs = pairs.sort_values(by=['pair'],  ascending=True)
        pairs = pairs[pairs['pair'] >= (group_size-1)]
groups

[[913, 363, 925, 796, 58],
 [461, 393, 844, 546, 933],
 [255, 938, 909, 366, 111],
 [277, 211, 643, 538, 724],
 [806, 533, 898, 826, 318],
 [766, 899, 307, 607, 204],
 [832, 127, 845, 841, 572],
 [755, 903, 655, 694, 42],
 [940, 801, 616, 848, 732],
 [864, 907, 194, 35, 603],
 [689, 833, 696, 623, 509],
 [671, 12, 734, 483, 353],
 [590, 3, 676, 19, 161],
 [737, 856, 855, 713, 619],
 [274, 173, 818, 811, 695],
 [209, 480, 417, 352, 253],
 [702, 495, 313, 343, 701],
 [26, 642, 14, 98, 243],
 [54, 242, 934, 677, 778],
 [877, 465, 900, 871, 545],
 [511, 342, 554, 237, 486],
 [291, 336, 715, 473, 678],
 [218, 628, 759, 52, 152],
 [0, 741, 757, 31, 860],
 [727, 842, 171, 517, 477],
 [102, 669, 725, 203, 376],
 [198, 795, 224, 268, 432],
 [490, 147, 779, 914, 191],
 [567, 201, 322, 217, 452],
 [210, 620, 448, 890, 722],
 [675, 258, 162, 704, 880],
 [522, 226, 613, 565, 599],
 [854, 325, 651, 178, 40],
 [719, 458, 340, 816, 357],
 [4, 501, 438, 314, 282],
 [82, 367, 419, 549, 712],
 [378, 661,

In [26]:
for group in groups:
    if len(group) != group_size:
        print('Group size error')
    for i in range(group_size):
        for j in range(i+1, group_size):
            u1 = group[i]
            u2 = group[j]
            c, p = pearson_correlation(u1, u2)
            if p > low_similarity_v:
                print('Similarity error', u1, u2, p)
user_status = np.zeros(num_user)
for group in groups:
    for u in group:
        user_status[u] += 1
for i in user_status:
    if i > 1:
        print('Depulicate error')

In [27]:
len(groups)

78

In [28]:
dir_ = '../data/groups/low/'
np.savetxt(os.path.join(dir_, 'group_' + str(group_size)  + '.csv'), groups, delimiter=",")